In [1]:
import pypsa
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
n=pypsa.Network()

#Snapshots
n.set_snapshots(value=pd.date_range(freq="h", start="2013-01-01", end="2014-01-01", inclusive='left'))
n.snapshot_weightings[:] *= 8760.0 / n.snapshot_weightings.sum()
Nyears = n.snapshot_weightings.objective.sum() / 8760.0

# Load yaml files
import yaml

with open(r'config.yaml') as file:
        config = yaml.load(file, Loader=yaml.FullLoader)
 
        print(config)

c:\Users\denis\miniconda3\envs\pypsa-earth\lib\site-packages\pypsa\components.py:374: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

c:\Users\denis\miniconda3\envs\pypsa-earth\lib\site-packages\pypsa\components.py:374: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

c:\Users\denis\miniconda3\envs\pypsa-earth\lib\site-packages\pypsa\components.py:374: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

c:\Users\denis\miniconda3\envs\pypsa-earth\lib\site-packages\pypsa\components.py:374: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

c:\Users\denis\miniconda3\envs\pypsa-earth\lib\site-packages\pypsa\components.py:374: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

c:\Users\denis\miniconda3\envs\pypsa-earth\lib\site-packages

{'version': '0.0.2', 'tutorial': True, 'logging': {'level': 'INFO', 'format': '%(levelname)s:%(name)s:%(message)s'}, 'scenario': {'simpl': [''], 'll': ['copt'], 'clusters': [6], 'opts': ['Co2L-4H']}, 'countries': ['NG'], 'summary_dir': 'results', 'snapshots': {'start': '2013-03-1', 'end': '2013-03-7', 'inclusive': 'left'}, 'enable': {'retrieve_databundle': False, 'download_osm_data': False, 'build_cutout': False, 'build_natura_raster': False}, 'crs': {'geo_crs': 'EPSG:4326', 'distance_crs': 'EPSG:3857', 'area_crs': 'ESRI:54009'}, 'retrieve_databundle': {'show_progress': False}, 'augmented_line_connection': {'add_to_snakefile': False, 'connectivity_upgrade': 2, 'new_line_type': ['HVAC'], 'min_expansion': 1, 'min_DC_length': 600}, 'cluster_options': {'alternative_clustering': False, 'distribute_cluster': ['load'], 'out_logging': True, 'aggregation_strategies': {'generators': {'p_nom': 'sum', 'p_nom_max': 'sum', 'p_nom_min': 'sum', 'p_min_pu': 'mean', 'marginal_cost': 'mean', 'committable

In [2]:
#I create the bus 

n.madd("Bus", ["onebus"], x=0.1, y=0.2, carrier="AC", v_nom=20)

Index(['onebus'], dtype='object')

In [3]:
costs=pd.read_csv(r'C:\Users\denis\OneDrive\Desktop\Mini grids\pypsa-distribution\costs.csv')

In [4]:
tech_costs=pd.read_csv(r'C:\Users\denis\OneDrive\Desktop\Mini grids\pypsa-distribution\costs.csv')

idx = pd.IndexSlice

def calculate_annuity(n, r):
    """
    Calculate the annuity factor for an asset with lifetime n years and
    discount rate of r, e.g. annuity(20, 0.05) * 20 = 1.6
    """
    if isinstance(r, pd.Series):
        return pd.Series(1 / n, index=r.index).where(
            r == 0, r / (1.0 - 1.0 / (1.0 + r) ** n)
        )
    elif r > 0:
        return r / (1.0 - 1.0 / (1.0 + r) ** n)
    else:
        return 1 / n

In [5]:
tech_costs='C://Users//denis//OneDrive//Desktop//Mini grids//pypsa-distribution//costs.csv'

In [6]:
def load_costs(tech_costs, config, elec_config, Nyears=1):
    """
    set all asset costs and other parameters
    """
    costs = pd.read_csv(tech_costs, index_col=list(range(3))).sort_index()

    # correct units to MW and EUR
    costs.loc[costs.unit.str.contains("/kW"), "value"] *= 1e3
    costs.loc[costs.unit.str.contains("USD"), "value"] *= config["USD2013_to_EUR2013"]

    costs = (
        costs.loc[idx[:, config["year"], :], "value"]
        .unstack(level=2)
        .groupby("technology")
        .sum(min_count=1)
    )

    costs = costs.fillna(
        {
            "CO2 intensity": 0,
            "FOM": 0,
            "VOM": 0,
            "discount rate": config["discountrate"],
            "efficiency": 1,
            "fuel": 0,
            "investment": 0,
            "lifetime": 25,
        }
    )

    costs["capital_cost"] = (
        (
            calculate_annuity(costs["lifetime"], costs["discount rate"])
            + costs["FOM"] / 100.0
        )
        * costs["investment"]
        * Nyears
    )

    costs.at["OCGT", "fuel"] = costs.at["gas", "fuel"]
    costs.at["CCGT", "fuel"] = costs.at["gas", "fuel"]

    costs["marginal_cost"] = costs["VOM"] + costs["fuel"] / costs["efficiency"]

    costs = costs.rename(columns={"CO2 intensity": "co2_emissions"})

    costs.at["OCGT", "co2_emissions"] = costs.at["gas", "co2_emissions"]
    costs.at["CCGT", "co2_emissions"] = costs.at["gas", "co2_emissions"]

    costs.at["solar", "capital_cost"] = 0.5 * (
        costs.at["solar-rooftop", "capital_cost"]
        + costs.at["solar-utility", "capital_cost"]
    )

    def costs_for_storage(store, link1, link2=None, max_hours=1.0):
        capital_cost = link1["capital_cost"] + max_hours * store["capital_cost"]
        if link2 is not None:
            capital_cost += link2["capital_cost"]
        return pd.Series(
            dict(capital_cost=capital_cost, marginal_cost=0.0, co2_emissions=0.0)
        )

    max_hours = elec_config["max_hours"]
    costs.loc["battery"] = costs_for_storage(
        costs.loc["battery storage"],
        costs.loc["battery inverter"],
        max_hours=max_hours["battery"],
    )
    costs.loc["H2"] = costs_for_storage(
        costs.loc["hydrogen storage"],
        costs.loc["fuel cell"],
        costs.loc["electrolysis"],
        max_hours=max_hours["H2"],
    )

    for attr in ("marginal_cost", "capital_cost"):
        overwrites = config.get(attr)
        if overwrites is not None:
            overwrites = pd.Series(overwrites)
            costs.loc[overwrites.index, attr] = overwrites

    return costs

In [7]:
costs = load_costs(
    tech_costs,
    config["costs"],
    config["electricity"],
    Nyears,
    )

In [8]:
def attach_wind_and_solar(n, costs, technologies, extendable_carriers):
    
    for tech in technologies:
        
        # with xr.open_dataset(technologies_path[tech]) as ds:
        with xr.open_dataset(f"renewable_profiles/profile_{tech}.nc") as ds:
            
            if ds.indexes["bus"].empty:
                continue   

            suptech = tech.split("-", 2)[0]
            
            n.madd(
            "Generator",
            ds.indexes["bus"],
            " " + tech,
            bus="onebus",
            carrier=tech,
            p_nom_extendable=tech in extendable_carriers["Generator"],
            p_nom_max=ds["p_nom_max"].to_pandas(), #guardare il config 
            weight=ds["weight"].to_pandas(),
            marginal_cost=costs.at[suptech, "marginal_cost"],
            capital_cost=costs.at[tech, "capital_cost"],
            efficiency=costs.at[suptech, "efficiency"],
            p_set=ds["profile"].transpose("time", "bus").to_pandas().reindex(n.snapshots),
            p_max_pu=ds["profile"].transpose("time", "bus").to_pandas().reindex(n.snapshots),
            )


In [9]:
attach_wind_and_solar(
    n,
    costs,
    config["renewable"],
    config["electricity"]["extendable_carriers"],
    )

In [10]:

#I create the load file
import numpy as np
p_set=np.random.rand(len(n.snapshots),(10))
array=p_set*100

print(array)

[[90.28691677 87.60412257 71.03497692 ... 62.31508464 50.65385483
  11.16498912]
 [ 6.08077385 25.81477288 26.23714862 ... 94.23153121 71.47537851
  15.11983112]
 [81.47590715 12.47740533 49.07696657 ...  1.35194612 14.53709631
  47.97661966]
 ...
 [37.4999401  52.68197801 84.01982143 ... 57.14684552 87.2052723
   1.42000019]
 [30.83700116 46.25809517 30.62878024 ... 29.75833261 34.67922151
  30.07300162]
 [27.60903802 70.62149979 54.73348479 ... 57.42573659 15.41433288
  10.62434254]]


In [11]:
import pandas as pd
df=pd.DataFrame(array)

filepath = 'my_file.xlsx'

df.to_excel(filepath, index=False)



In [12]:
load_df=pd.read_excel(r'C:\Users\denis\OneDrive\Desktop\Mini grids\pypsa-distribution\my_file.xlsx')
# load_df=load_df.set_index([n.snapshots])
# print(load_df)


In [13]:
def attach_load(n, load_paths):
    
    for col in load_df:

        n.madd("Load", [f"load{col}"] , bus=["onebus"], carrier="AC", p_set=load_df[col])



In [14]:
load_paths=r'C:\Users\denis\OneDrive\Desktop\Mini grids\pypsa-distribution\my_file.xlsx'

In [15]:
attach_load(n, load_paths)

In [23]:
print(n)

PyPSA Network
Components:
 - Bus: 1
 - Generator: 449
 - Load: 10
 - SubNetwork: 1
Snapshots: 8760


In [20]:
# Optimization
from pypsa.linopf import ilopf

solver_name="gurobi"

n.lopf(n.snapshots, solver_name=solver_name, pyomo=False)

INFO:pypsa.linopf:Prepare linear problem
INFO:pypsa.linopf:Total preparation time: 48.08s
INFO:pypsa.linopf:Solve linear problem using Gurobi solver


Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-07
Read LP format model from file C:\Users\denis\AppData\Local\Temp\pypsa-problem-4xwlsyc9.lp
Reading time = 28.36 seconds
obj: 7875240 rows, 3933690 columns, 11838308 nonzeros
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 7875240 rows, 3933690 columns and 11838308 nonzeros
Model fingerprint: 0xf06e8be0
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [1e-02, 1e+05]
  Bounds range     [3e-01, 2e+05]
  RHS range        [0e+00, 0e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 7836508 rows and 1 columns (presolve time = 7s) ...
Presolve removed 7875240 rows and 3933690 columns (presolve time = 11s) ...
Presolve removed 7875240 rows and 3933690 columns (presolve time = 16s) ...
Presolve removed 7875240 rows and 39336

INFO:pypsa.linopf:Optimization successful. Objective value: 0.00e+00


('ok', 'optimal')